#  Unit 2.4b Using Programs with Data, SQL (Notebook FOR HACKS)
> Using Programs with Data is focused on SQL and database actions.  Part B focuses on learning SQL commands, connections, and curses using an Imperative programming style,
- toc: true
- image: /images/python.png
- categories: [Notes]
- type: ap
- week: 26

In [2]:
import sqlite3

database = 'instance/sqlite.db' # this is location of database

def schema():
    
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    
    # Fetch results of Schema
    results = cursor.execute("PRAGMA table_info('users')").fetchall()

    # Print the results
    for row in results:
        print(row)

    # Close the database connection
    conn.close()
    
schema()


(0, 'id', 'INTEGER', 1, None, 1)
(1, '_name', 'VARCHAR(255)', 1, None, 0)
(2, '_uid', 'VARCHAR(255)', 1, None, 0)
(3, '_password', 'VARCHAR(255)', 1, None, 0)
(4, '_dob', 'DATE', 0, None, 0)


## Reading Users table in Sqlite.db
> Uses SQL SELECT statement to read data
- Schema is the Metadata for the databse
- What is a connection object?  After you google it, what do you think it does?
    - This conn is an object that has variables and functions. 
    - I think that this connection instantiates an obkect in order to connect our databse to a variable.
    - We connect to the database using SQLite3.connect
- Same for cursor object?
    - 
- Look at conn object and cursor object in VSCode debugger.  What attributes are in the object?
- Is "results" an object?  How do you know?

In [1]:
import sqlite3

def read():
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    
    # Execute a SELECT statement to retrieve data from a table
    results = cursor.execute('SELECT * FROM etrack_users').fetchall()

    # Print the results
    if len(results) == 0:
        print("Table is empty")
    else:
        for row in results:
            print(row)

    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
read()


NameError: name 'database' is not defined

## Create a new User in table in Sqlite.db
> Uses SQL INSERT to add row
-  Compore create() in both SQL lessons.  What is better or worse in the two implementations?
-  Explain purpose of SQL INSERT.   Is this the same as User __init__?

In [11]:
import sqlite3

def create():
    name = input("Enter your name:")
    uid = input("Enter your user id:")
    password = input("Enter your password")
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    try:
        # Execute an SQL command to insert data into a table
        cursor.execute("INSERT INTO users (_name, _uid, _password, _dob) VALUES (?, ?, ?, ?)", (name, uid, password, dob))
        
        # Commit the changes to the database
        conn.commit()
        print(f"A new user record {uid} has been created")
                
    except sqlite3.Error as error:
        print("Error while executing the INSERT:", error)


    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
#create()

## Updating a User in table in Sqlite.db
> Uses SQL UPDATE to modify password
- What does the hacked part do?
- Explain try/except, when would except occur?
- What code seems to be repeated in each of these examples to point, why is it repeated?

In [7]:
import sqlite3

def update():
    uid = input("Enter user id to update")
    password = input("Enter updated password")
    if len(password) < 2:
        message = "hacked"
        password = 'gothackednewpassword123'
    else:
        message = "successfully updated"

    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    try:
        # Execute an SQL command to update data in a table
        cursor.execute("UPDATE users SET _password = ? WHERE _uid = ?", (password, uid))
        if cursor.rowcount == 0:
            # The uid was not found in the table
            print(f"No uid {uid} was not found in the table")
        else:
            print(f"The row with user id {uid} the password has been {message}")
            conn.commit()
    except sqlite3.Error as error:
        print("Error while executing the UPDATE:", error)
        
    
    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
#update()


## Delete a User in table in Sqlite.db
> Uses a delete function to remove a user based on a user input of the id.
- Is DELETE a dangerous operation?  Why?
- In the print statemements, what is the "f" and what does {uid} do?

In [8]:
import sqlite3

def delete():
    uid = input("Enter user id to delete")

    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()
    
    try:
        cursor.execute("DELETE FROM users WHERE _uid = ?", (uid,))
        if cursor.rowcount == 0:
            # The uid was not found in the table
            print(f"No uid {uid} was not found in the table")
        else:
            # The uid was found in the table and the row was deleted
            print(f"The row with uid {uid} was successfully deleted")
        conn.commit()
    except sqlite3.Error as error:
        print("Error while executing the DELETE:", error)
        
    # Close the cursor and connection objects
    cursor.close()
    conn.close()
    
#delete()

# Menu Interface to CRUD operations
> CRUD and Schema interactions from one location by running menu. Observe input at the top of VSCode, observe output underneath code cell.
- Why does the menu repeat?
- Could you refactor this menu?  Make it work with a List?

In [12]:
# Menu, to run other cells from one control point
def menu():
    operation = input("Enter: (C)reate (R)ead (U)pdate or (D)elete or (S)chema")
    if operation.lower() == 'c':
        create()
    elif operation.lower() == 'r':
        read()
    elif operation.lower() == 'u':
        update()
    elif operation.lower() == 'd':
        delete()
    elif operation.lower() == 's':
        schema()
    elif len(operation)==0: # Escape Key
        return
    else:
        print("Please enter c, r, u, or d") 
    menu() # recursion, repeat menu
        
try:
    menu() # start menu
except:
    print("Perform Jupyter 'Run All' prior to starting menu")


(1, 'Thomas Edison', 'toby', 'sha256$z8JBP2pU25i597RO$217ec7f4593df3b5aaa184f58bb65dc080cdd1cb977c60455f92c307a18970fe', '1847-02-11')
(2, 'Nikola Tesla', 'niko', 'sha256$8DMUyZfuXnzIQJeJ$87ab31383e3733f7b13da8266b0f1d7b36fb0af6869718cab68493512490abcf', '2023-03-15')
(3, 'Alexander Graham Bell', 'lex', 'sha256$3mD0XNHjHB9M2C51$d03ecd111724922b57f89006d586f6972279a76a17b803337230563ea45b6828', '2023-03-15')
(4, 'Eli Whitney', 'whit', 'sha256$xOCEamJvuHizbbxU$4c0f6f0391dc28cf820864f4de4382ffb3b6cb41f6cc09f044b8cab7f2a9e0a3', '2023-03-15')
(5, 'Indiana Jones', 'indi', 'sha256$mjRhVRtaxg9CT6IR$0269c149d4ca6bbcc49f3989b8d2f47b4cd28f44e6ae747cb42c01f2699b7f44', '1920-10-21')
(6, 'Marion Ravenwood', 'raven', 'sha256$8rf6nuMMzmIolZ31$c642d4e35f95c72e61fd558c5b95ea0c7b7203c3a112eeaa14d84195242fb588', '1921-10-21')
(7, 'Liav B', 'LaveBear2', 'SuperMort34', '2006-08-09')
(1, 'Thomas Edison', 'toby', 'sha256$z8JBP2pU25i597RO$217ec7f4593df3b5aaa184f58bb65dc080cdd1cb977c60455f92c307a18970fe', '1847